# Modelos de Regresion Espacial SAR

## Autocorrelación Endógena con la variable dependiente

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

In [2]:
gdf = gpd.read_file("https://github.com/algarciach/AnalisisGeoespacial/raw/main/Covid19_model/Data/covid19_municipios_antioquia.gpkg")
gdf.info()

In [24]:
gdf['density_cfr'] = (gdf['cfr'] / gdf['area_municipio'])
gdf['y'] = np.log(gdf['density_cfr'] + 1)

y = gdf['y'].values.reshape(-1, 1)
independent_vars = ["altitud", "temperatura", "humedad_relativa", "urbanizacion", "densidad"]
X = gdf[independent_vars].values

st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)
w = Queen.from_dataframe(gdf)

wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

ols_model = spreg.OLS(gdf['y'].values.reshape((-1, 1)), 
                      slx_exog.values, 
                      name_y='log_density_cfr', 
                      name_x=slx_exog.columns.tolist(),
                      name_w='queen_contiguity')

print(ols_model.summary)

/var/folders/nq/kj34bm5140bgwbktdx1z94cm0000gn/T/ipykernel_1010/1856113932.py:11: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :log_density_cfr                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :          11
S.D. dependent var  :      0.0161                Degrees of Freedom    :         114
R-squared           :      0.2723
Adjusted R-squared  :      0.2085
Sum squared residual:    0.023518                F-statistic           :      4.2656
Sigma-square        :       0.000                Prob(F-statistic)     :   4.707e-05
S.E. of regression  :       0.014                Log likelihood        :     358.777
Sigma-square ML     :       0.000                Akaike info criterion :    -695.553
S.E of regression ML:      0.0137                Schwarz criterion     :    -664.442

---------------------------------------------------------

In [25]:
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w,
                                         name_y='log_density_cfr_rec',
                                         name_x=independent_vars,
                                         name_w='queen_contiguity')
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           6
S.D. dependent var  :      0.0161                Degrees of Freedom    :         119
Pseudo R-squared    :      0.2464
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01605         0.00193         8.33170         0.00000
             altitud         0.02175         

In [26]:
sem = spreg.ML_Error(y, X, w=w, name_y='log_density_cfr_rec', name_x=independent_vars, name_w='queen_contiguity')
print(sem.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           6
S.D. dependent var  :      0.0161                Degrees of Freedom    :         119
Pseudo R-squared    :      0.2458
Log likelihood      :    358.0124
Sigma-square ML     :      0.0002                Akaike info criterion :    -704.025
S.E of regression   :      0.0137                Schwarz criterion     :    -687.055

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -

/usr/local/Caskroom/miniforge/base/envs/geo/lib/python3.11/site-packages/spreg/ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(


In [27]:
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w,
                                 name_y='log_density_cfr_rec',
                                 name_x=independent_vars,
                                 name_w='queen_contiguity')
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           7
S.D. dependent var  :      0.0161                Degrees of Freedom    :         118
Pseudo R-squared    :      0.2489
Spatial Pseudo R-squared:  0.2504

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01473         0.00419         3.51910         0.00043
             altitud         0.01809         0.01059         1.70716         0.08779
         temperatura         0.